Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS310.M11.1

Buổi 01 - 30/09/2021

# LAB 01 - PHÂN LỚP VĂN BẢN CƠ BẢN

## Naive Bayes cơ bản

### Bộ dữ liệu 1

In [38]:
train = [["just plain boring", "-"], 
         ["entirely predictable and lacks energy", "-"],
         ["no surprises and very few laughs", "-"], 
         ["very powerful", "+"], 
         ["the most fun film of the summer", "+"]] 
 
test = "predictable with no fun" 

### Bộ dữ liệu 2

In [39]:
train2 = [["sản_phẩm A rất tốt.", "+"], 
         ["tôi không thích sản_phẩm A.", "-"], 
         ["màu_sắc của A thật tốt.", "+"], 
         ["sản_phẩm A thật kinh_khủng.", "-"]] 
 
test2 = ["sản_phẩm A khá tốt.", "màu_sắc quá kinh_khủng."]

### Xây dựng model theo công thức NAIVE BAYES

In [40]:
Word_Positive = [] # Tập các từ có nhãn là '+'
Word_Negative = [] # Tập các từ có nhãn là '-'
Word_Total = [] # Tập từ vựng 
y_train = [] # Tập các nhãn của các câu
Prior_Positive = 0 # Xác suất của nhã '+'
Prior_Negative = 0 # Xác suất của nhãn '-'

for s in train:
  if s[1] == '+':
    Word_Positive.extend(s[0].split())
    Prior_Positive += 1
  else:
    Word_Negative.extend(s[0].split())
    Prior_Negative += 1
  y_train.append(s[1])

Word_Total = set(Word_Positive + Word_Negative)

Count_Negative = len(Word_Negative) # Số lượng các từ có nhãn '-'
Count_Positive = len(Word_Positive) # Số lượng các từ có nhãn '+'
Count_Total = len(Word_Total) # Tổng số lượng từ khác nhau có trong tập từ vựng
Prior_Positive /= len(y_train)
Prior_Negative /= len(y_train)

In [41]:
# Drop 'with'
test_new = test.split()
test_new.remove('with')

In [42]:
# Likelihoods from training
P_Negative = []
P_Positive = []

for i in test_new:
  a = Word_Negative.count(i)
  b = Word_Positive.count(i)
  P_Negative.append((a+1)/(Count_Negative+Count_Total))
  P_Positive.append((b+1)/(Count_Positive+Count_Total))

In [43]:
# Scoring the test set
P_test_Positive = Prior_Positive
P_test_Negative = Prior_Negative

for i in range(len(test_new)):
  P_test_Positive *= P_Positive[i]
  P_test_Negative *= P_Negative[i]

print('P(+)P(S|+) =', P_test_Positive)
print('P(-)P(S|-) =', P_test_Negative)

if P_test_Positive > P_test_Negative:
  print('The class of "{}" is +'.format(test))
else:
  print('The class of "{}" is -'.format(test))

P(+)P(S|+) = 3.2801672885317154e-05
P(-)P(S|-) = 6.106248727864848e-05
The class of "predictable with no fun" is -


### Xây dựng model sử dụng thư viện sklearn.

#### Import thư viện cần thiết

In [44]:
import pandas as pd
import numpy as np


from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

#### Xử lý dữ liệu

In [45]:
X_train = []
y_train = []
X_test = [test]

for s in train:
  X_train.append(s[0])
  y_train.append(s[1])

#### Mã hoá văn bản

In [46]:
cv = CountVectorizer()
cv.fit(X_train)
X_train_new = cv.transform(X_train)

In [47]:
cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['and',
 'boring',
 'energy',
 'entirely',
 'few',
 'film',
 'fun',
 'just',
 'lacks',
 'laughs',
 'most',
 'no',
 'of',
 'plain',
 'powerful',
 'predictable',
 'summer',
 'surprises',
 'the',
 'very']

In [48]:
X_test_new = cv.transform(X_test)

#### Mã hoá nhãn

In [49]:
le = LabelEncoder()
le.fit(y_train)
y_train_new = le.transform(y_train)

In [50]:
y_train_new

array([1, 1, 1, 0, 0])

#### Training model

In [51]:
clf = MultinomialNB()
clf.fit(X_train_new, y_train_new)

MultinomialNB()

#### Predict

In [52]:
y_pred = clf.predict(X_test_new)

In [53]:
print('The class of "{}" is {}'.format(test, le.inverse_transform(y_pred)[0]))

The class of "predictable with no fun" is -


## N-gram

Xây dựng model với 2-gram

#### Mã hoá văn bản

In [54]:
cv2 = CountVectorizer(analyzer='word', ngram_range=(2,2))
cv2.fit(X_train)
X_train_new2 = cv2.transform(X_train)

In [55]:
cv2.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['and lacks',
 'and very',
 'entirely predictable',
 'few laughs',
 'film of',
 'fun film',
 'just plain',
 'lacks energy',
 'most fun',
 'no surprises',
 'of the',
 'plain boring',
 'predictable and',
 'surprises and',
 'the most',
 'the summer',
 'very few',
 'very powerful']

In [56]:
X_test_new2 = cv2.transform(X_test)

#### Mã hoá nhãn

In [57]:
le = LabelEncoder()
le.fit(y_train)
y_train_new2 = le.transform(y_train)

In [58]:
y_train_new2

array([1, 1, 1, 0, 0])

#### Training model

In [59]:
clf2 = MultinomialNB()
clf2.fit(X_train_new2, y_train_new2)

MultinomialNB()

#### Predict

In [60]:
y_pred2 = clf2.predict(X_test_new2)

In [61]:
print('The class of "{}" is {}'.format(test, le.inverse_transform(y_pred2)[0]))

The class of "predictable with no fun" is -


**Nhận xét**: Kết quả giống với model 1-gram

## Bài tập vận dụng

### Bài tập 1: Hiện thực lại mô hình Naive Bayes bằng tay cho Bộ dữ liệu 1 và Bộ dữ liệu 2.

#### Hiện thực lại mô hình Naive Bayes bằng tay cho Bộ dữ liệu 1

In [62]:
Word_Positive = [] # Tập các từ có nhãn là '+'
Word_Negative = [] # Tập các từ có nhãn là '-'
Word_Total = [] # Tập từ vựng 
y_train = [] # Tập các nhãn của các câu
Prior_Positive = 0 # Xác suất của nhã '+'
Prior_Negative = 0 # Xác suất của nhãn '-'

for s in train:
  if s[1] == '+':
    Word_Positive.extend(s[0].split())
    Prior_Positive += 1
  else:
    Word_Negative.extend(s[0].split())
    Prior_Negative += 1
  y_train.append(s[1])

Word_Total = set(Word_Positive + Word_Negative)

Count_Negative = len(Word_Negative) # Số lượng các từ có nhãn '-'
Count_Positive = len(Word_Positive) # Số lượng các từ có nhãn '+'
Count_Total = len(Word_Total) # Tổng số lượng từ khác nhau có trong tập từ vựng
Prior_Positive /= len(y_train)
Prior_Negative /= len(y_train)

In [63]:
# Drop 'with'
test_new = test.split()
test_new.remove('with')

In [64]:
# Likelihoods from training
P_Negative = []
P_Positive = []

for i in test_new:
  a = Word_Negative.count(i)
  b = Word_Positive.count(i)
  P_Negative.append((a+1)/(Count_Negative+Count_Total))
  P_Positive.append((b+1)/(Count_Positive+Count_Total))

In [65]:
# Scoring the test set
P_test_Positive = Prior_Positive
P_test_Negative = Prior_Negative

for i in range(len(test_new)):
  P_test_Positive *= P_Positive[i]
  P_test_Negative *= P_Negative[i]

print('P(+)P(S|+) =', P_test_Positive)
print('P(-)P(S|-) =', P_test_Negative)

if P_test_Positive > P_test_Negative:
  print('The class of "{}" is +'.format(test))
else:
  print('The class of "{}" is -'.format(test))

P(+)P(S|+) = 3.2801672885317154e-05
P(-)P(S|-) = 6.106248727864848e-05
The class of "predictable with no fun" is -


#### Hiện thực lại mô hình Naive Bayes bằng tay cho Bộ dữ liệu 2

In [66]:
Word_Positive2 = [] # Tập các từ có nhãn là '+'
Word_Negative2 = [] # Tập các từ có nhãn là '-'
Word_Total2 = [] # Tập từ vựng 
y_train2 = [] # Tập các nhãn của các câu
Prior_Positive2 = 0 # Xác suất của nhã '+'
Prior_Negative2 = 0 # Xác suất của nhãn '-'

for s in train2:
  if s[1] == '+':
    Word_Positive2.extend(s[0].split())
    Prior_Positive2 += 1
  else:
    Word_Negative2.extend(s[0].split())
    Prior_Negative2 += 1
  y_train2.append(s[1])

Word_Total2 = set(Word_Positive2 + Word_Negative2)

Count_Negative2 = len(Word_Negative2) # Số lượng các từ có nhãn '-'
Count_Positive2 = len(Word_Positive2) # Số lượng các từ có nhãn '+'
Count_Total2 = len(Word_Total2) # Tổng số lượng từ khác nhau có trong tập từ vựng
Prior_Positive2 /= len(y_train2)
Prior_Negative2 /= len(y_train2)

In [67]:
test_new2 = []
for s in test2:
  test_new2.append(s.split())

In [68]:
# Likelihoods from training
P_Negative2 = []
P_Positive2 = []

for s in test_new2:
  t1 = []
  t2 = []
  for i in range(len(s)):
    a = Word_Negative2.count(s[i])
    b = Word_Positive2.count(s[i])
    t1.append((a+1)/(Count_Negative2+Count_Total2))
    t2.append((b+1)/(Count_Positive2+Count_Total2))
  P_Negative2.append(t1)
  P_Positive2.append(t2)

In [69]:
# Scoring the test set
P_test_Positive2 = [Prior_Positive2]*2
P_test_Negative2 = [Prior_Negative2]*2

for i in range(len(test_new2)):
  for j in range(len(test_new2[i])):
    P_test_Positive2[i] *= P_Positive2[i][j]
    P_test_Negative2[i] *= P_Negative2[i][j]


for i in range(len(test_new2)):
  print('P(+)P(S|+) =', P_test_Positive2[i])
  print('P(-)P(S|-) =', P_test_Negative2[i])
  if P_test_Positive2[i] > P_test_Negative2[i]:
    print('The class of "{}" is +'.format(test2[i]))
  else:
    print('The class of "{}" is -'.format(test2[i]))

P(+)P(S|+) = 4.627701420704335e-05
P(-)P(S|-) = 1.5425671402347783e-05
The class of "sản_phẩm A khá tốt." is +
P(+)P(S|+) = 0.00010797969981643449
P(-)P(S|-) = 0.00010797969981643449
The class of "màu_sắc quá kinh_khủng." is -


### Bài tập 2: Hiện thực lại mô hình Naive Bayes bằng thư viện sklearn cho Bộ dữ liệu 1 và  Bộ dữ liệu 2. (Sử dụng CountVectorizer 1-gram) 

#### Hiện thực lại mô hình Naive Bayes bằng thư viện sklearn cho Bộ dữ liệu 1

##### Import thư viện cần thiết

In [70]:
import pandas as pd
import numpy as np


from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

##### Xử lý dữ liệu

In [71]:
X_train = []
y_train = []
X_test = [test]

for s in train:
  X_train.append(s[0])
  y_train.append(s[1])

##### Mã hoá văn bản

In [72]:
cv = CountVectorizer()
cv.fit(X_train)
X_train_new = cv.transform(X_train)

In [73]:
cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['and',
 'boring',
 'energy',
 'entirely',
 'few',
 'film',
 'fun',
 'just',
 'lacks',
 'laughs',
 'most',
 'no',
 'of',
 'plain',
 'powerful',
 'predictable',
 'summer',
 'surprises',
 'the',
 'very']

In [74]:
X_test_new = cv.transform(X_test)

##### Mã hoá nhãn

In [75]:
le = LabelEncoder()
le.fit(y_train)
y_train_new = le.transform(y_train)

In [76]:
y_train_new

array([1, 1, 1, 0, 0])

##### Training model

In [77]:
clf = MultinomialNB()
clf.fit(X_train_new, y_train_new)

MultinomialNB()

##### Predict

In [78]:
y_pred = clf.predict(X_test_new)

In [79]:
print('The class of "{}" is {}'.format(test, le.inverse_transform(y_pred)[0]))

The class of "predictable with no fun" is -


#### Hiện thực lại mô hình Naive Bayes bằng thư viện sklearn cho Bộ dữ liệu 2

##### Import thư viện cần thiết

In [80]:
import pandas as pd
import numpy as np


from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

##### Xử lý dữ liệu

In [81]:
X_train2 = []
y_train2 = []
X_test2 = test2

for s in train2:
  X_train2.append(s[0])
  y_train2.append(s[1])

##### Mã hoá văn bản

In [82]:
cv = CountVectorizer()
cv.fit(X_train2)
X_train_new2 = cv.transform(X_train2)

In [83]:
cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['của',
 'không',
 'kinh_khủng',
 'màu_sắc',
 'rất',
 'sản_phẩm',
 'thích',
 'thật',
 'tôi',
 'tốt']

In [84]:
X_test_new2 = cv.transform(X_test2)

##### Mã hoá nhãn

In [85]:
le = LabelEncoder()
le.fit(y_train2)
y_train_new2 = le.transform(y_train2)

In [86]:
y_train_new2

array([0, 1, 0, 1])

##### Training model

In [87]:
clf = MultinomialNB()
clf.fit(X_train_new2, y_train_new2)

MultinomialNB()

##### Predict

In [88]:
y_pred = clf.predict(X_test_new2)

In [89]:
print('The class of "{}" is {}'.format(test2[0], le.inverse_transform(y_pred)[0][0]))
print('The class of "{}" is {}'.format(test2[1], le.inverse_transform(y_pred)[1][0]))

The class of "sản_phẩm A khá tốt." is +
The class of "màu_sắc quá kinh_khủng." is +
